In [138]:
%pylab inline
from scipy.linalg import hadamard as hadamard
from sympy import isprime as isprime
from math import gcd as GCD
from scipy.sparse.coo import coo_matrix as sparsemat
from scipy.sparse.linalg import svds as sparsesvd
from itertools import combinations

Populating the interactive namespace from numpy and matplotlib


/home/simone/anaconda3/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['test']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [139]:
def decimal_to_state(m,nqubit):
    '''
        Return binary representation of m as array of nqubit qubits

            Parameters
            ----------                
            m:   Integer
                number to be representend in binary form
                
            nqubit: Integer
                Total number of qubits used for representation
                    
    '''
    
    arr=binary_repr(m)
    arr=[int(i) for i in arr]
    if(len(arr)>nqubit):
        raise ValueError(str(nqubit)+" are not enough qubits to store the number "+str(m))
    if(len(arr)==nqubit): return arr
    return list(np.zeros(nqubit-len(arr),dtype=int16))+ arr


def to_decimal(array):
    '''
        Return decimal representation of the array storing a number in binary form

        Example: input [1,0,1,0,1] returns 21 
            Parameters
            ----------                
            array: List
                array containing binary representation of a number
                
                    
    '''

    size=len(array)
    return int(np.sum([array[i] *2**(size-i-1)  for i in range(size)]))

def find_coprime(N):     
    """
            Find a coprime of N for N>2
            Parameters
            ----------
            N:  Integer
                Number to find the coprime of

    """
    if(N<3):
        raise ValueError("Illegal argument: coprimes exist for N>2")
    Y=randint(2,N)
    used=[]
    while 1:
        a=GCD(Y,N)
        if (a>1):
            #this Y is not coprime
            used.append(Y)
            while(Y in used):
                Y=randint(2,N)
        else: return Y

In [140]:
def notchosen(chosen,system_size):
    """
            Return array containing the qubit NOT in the partition
            
            Parameters
            ----------                
            chosen:   List
                List of the qubits selected as partition, in the form [1,3,7,..]
                
            system_size: Integer
                Total number of qubits
                    
    """
    
    notchosen=list(set(list(range(system_size)))-set(chosen))
    notchosen.sort()
    return notchosen

def create_W(k,Y,N,chosen):
    '''
    creates W directly
    
    '''
    
    L=int(ceil(log2(N)))
    if(k>2*L):
        raise ValueError(str(k)+"th computational step does not make sense in a "+str(2*L)+" qubits control register")
    
    #nonzero elements of psi in binary form
    nonzeros=[decimal_to_state(m*2**L+(Y**m%N),k+L) for m in range(2**k)]
    not_chosen=notchosen(chosen,k+L)  

    indexes=[ (to_decimal(split_components(i,chosen)),to_decimal((split_components(i,not_chosen)))) for i in nonzeros]
    row=[elem[0] for elem in indexes]
    col=[elem[1] for elem in indexes]
    data=np.ones(2**k)/sqrt(2**k)
    
    return sparsemat((data,(row,col)), shape=(2**len(chosen),2**len(not_chosen))    ).tocsc()

def split_components(array,chosen):
    '''
    Given an input array and selected components returns two arrays.
    The first array contains only the chosen components and the other the remainders
    '''
    if( max(chosen) not in range(len(array)))  :
        raise ValueError('the chosen '+str(max(chosen))+' bit is not present in a '+str(len(array))+' bits register')
    return [array[i] for i in chosen]


def entanglement_entropy(k,Y,N,chosen):
    W=create_W(k,Y,N,chosen)
    eigs=sparsesvd(W,k=min(shape(W))-1,which='LM',return_singular_vectors=False)
    eigs=eigs*eigs
    return -np.sum([i*np.log2(i) for i in eigs if i>0])

In [196]:
N=3
L=int(ceil(log2(N)))
#Y=find_coprime(N)
#print(Y)
Y= #N,Y=(3,2),(6,5),(10,7),(11,8), (64,57)

In [197]:
%%time
#times:(3..4,40 ms), (5..8,300ms)
#5.5 sec for N=9,..,16, 150s for N=17,..,32, not feasible for N=33,..,64

entropy=[entanglement_entropy(k,Y,N,chosen) for k in range(1,2*L) for chosen in combinations(range(k+L),int((k+L)/2))]
#entropy[0] will contain all computational steps of first chosen partition

CPU times: user 9 µs, sys: 1e+03 ns, total: 10 µs
Wall time: 12.6 µs
